# EquilibriumActor example

In [ ]:
using Revise
using FUSE
using Plots; gr();
global_logger(FUSE.logger);

### Initialize ITER case
[ITER case documentation](https://fuse.help/cases.html#ITER)

In [ ]:
dd, ini, act = FUSE.init(:ITER, init_from=:ods);
eq_plot = plot(dd.equilibrium, label="ITER ods equilibrium")

### Setting up the equlibrium actor
The equilibrium actor selects which equilibrium model is used, right now the option are :CHEASE & Solovev

In [ ]:
act.ActorEquilibrium.model = :CHEASE

### Comparing Solovev with CHEASE equilibrium to the original ITER ods equilibrium

In [ ]:
### CHEASE
FUSE.ActorEquilibrium(dd,act)
display(plot!(eq_plot,dd.equilibrium, label="CHEASE equilibrium"))

### Solovev
dd, ini, act = FUSE.init(:ITER, init_from=:ods);
eq_plot = plot(dd.equilibrium, label="ITER ods equilibrium")


### Starting from scalars and running CHEASE

In [ ]:
dd, ini, act = FUSE.init(:FPP,version=:v1_demount, init_from=:ods);
eq_plot = plot(dd.equilibrium, label="FPP ODS eq")

# CHEASE deadstart
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:FPP,version=:v1_demount, init_from=:scalars);
act.ActorEquilibrium.model = :CHEASE
dd = FUSE.init(dd,ini,act)
plot!(eq_plot, dd.equilibrium,label="CHEASE deadstart",color=:red)

# Solvev from scalars
ini, act = FUSE.case_parameters(:FPP,version=:v1_demount, init_from=:scalars);
act.ActorEquilibrium.model = :Solovev
dd= FUSE.init(dd,ini,act)
display(plot!(eq_plot, dd.equilibrium,label="Solvev deadstart"))

### Finding a MXH parameterization of the boundary for ITER

In [ ]:
import MillerExtendedHarmonic:flat_coeffs
dd, ini, act = FUSE.init(:ITER,init_from=:ods);
eqt = dd.equilibrium.time_slice[]
eqt_bound= eqt.boundary.outline
psi_close = (eqt.profiles_1d.psi[end-1] + 5 * eqt.profiles_1d.psi[end]) / 6 
r_bound,z_bound = IMAS.flux_surface(eqt,  psi_close)

eq_plot = plot(r_bound,z_bound,label="0.99 of original boundary shape")
for order in [6]
    mxh = IMAS.MXH(r_bound,z_bound,order)
    @show round.(flat_coeffs(mxh),digits=5)
    plot!(mxh,label="MXH $order",lw=0.3)
end
display(plot!(eqt_bound.r,eqt_bound.z,label="original boundary shape",color="black",marker="x",lw=0,markersize=1))

## Comparing CHEASE deadstart from MXH parameterization vs Miller parameterization

In [ ]:
dd, ini, act = FUSE.init(:ITER, init_from=:ods);
eq_plot = plot(dd.equilibrium, label="ITER ODS eq")
display(eq_plot)

# CHEASE deadstart MXH
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);
ini.equilibrium.boundary_from = :MXH_params
act.ActorEquilibrium.model = :CHEASE
dd = FUSE.init(dd,ini,act)
plot!(eq_plot, dd.equilibrium,label="CHEASE deadstart MXH")

# CHEASE deadstart MILLER
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);
ini.equilibrium.boundary_from = :miller
act.ActorEquilibrium.model = :CHEASE
act.ActorCHEASE.free_boundary=false
dd = FUSE.init(dd,ini,act)
display(plot!(eq_plot, dd.equilibrium,label="CHEASE deadstart Miller"))

